# <center> 【Kaggle】Telco Customer Churn 电信用户流失预测案例

---

## <font face="仿宋">第四部分导读

&emsp;&emsp;<font face="仿宋">在案例的第二、三部分中，我们详细介绍了关于特征工程的各项技术，特征工程技术按照大类来分可以分为数据预处理、特征衍生、特征筛选三部分，其中特征预处理的目的是为了将数据集整理、清洗到可以建模的程度，具体技术包括缺失值处理、异常值处理、数据重编码等，是建模之前必须对数据进行的处理和操作；而特征衍生和特征筛选则更像是一类优化手段，能够帮助模型突破当前数据集建模的效果上界。并且我们在第二部分完整详细的介绍机器学习可解释性模型的训练、优化和解释方法，也就是逻辑回归和决策树模型。并且此前我们也一直以这两种算法为主，来进行各个部分的模型测试。

&emsp;&emsp;<font face="仿宋">而第四部分，我们将开始介绍集成学习的训练和优化的实战技巧，尽管从可解释性角度来说，集成学习的可解释性并不如逻辑回归和决策树，但在大多数建模场景下，集成学习都将获得一个更好的预测结果，这也是目前效果优先的建模场景下最常使用的算法。

&emsp;&emsp;<font face="仿宋">总的来说，本部分内容只有一个目标，那就是借助各类优化方法，抵达每个主流集成学习的效果上界。换而言之，本部分我们将围绕单模优化策略展开详细的探讨，涉及到的具体集成学习包括随机森林、XGBoost、LightGBM、和CatBoost等目前最主流的集成学习算法，而具体的优化策略则包括超参数优化器的使用、特征衍生和筛选方法的使用、单模型自融合方法的使用，这些优化方法也是截至目前，提升单模效果最前沿、最有效、同时也是最复杂的方法。其中有很多较为艰深的理论，也有很多是经验之谈，但无论如何，我们希望能够围绕当前数据集，让每个集成学习算法优化到极限。值得注意的是，在这个过程中，我们会将此前介绍的特征衍生和特征筛选视作是一种模型优化方法，衍生和筛选的效果，一律以模型的最终结果来进行评定。而围绕集成学习进行海量特征衍生和筛选，也才是特征衍生和筛选技术能发挥巨大价值的主战场。

&emsp;&emsp;<font face="仿宋">而在抵达了单模的极限后，我们就会进入到下一阶段，也就是模型融合阶段。需要知道的是，只有单模的效果到达了极限，进一步的多模型融合、甚至多层融合，才是有意义的，才是有效果的。

---

# <center>Part 4.集成算法的训练与优化技巧

In [1]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

import warnings
warnings.filterwarnings('ignore')

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

# 自定义模块
from telcoFunc import *
# 导入特征衍生模块
import features_creation as fc
from features_creation import *

# re模块相关
import inspect, re

# 其他模块
from tqdm import tqdm
import gc

&emsp;&emsp;然后执行Part 1中的数据清洗相关工作：

In [2]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [3]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

&emsp;&emsp;同时，创建自然编码后的数据集以及经过时序特征衍生的数据集：

In [4]:
# 划分训练集和测试集
train, test = train_test_split(tcc, random_state=22)

X_train = train.drop(columns=[ID_col, target]).copy()
X_test = test.drop(columns=[ID_col, target]).copy()

y_train = train['Churn'].copy()
y_test = test['Churn'].copy()

X_train_seq = pd.DataFrame()
X_test_seq = pd.DataFrame()

# 年份衍生
X_train_seq['tenure_year'] = ((72 - X_train['tenure']) // 12) + 2014
X_test_seq['tenure_year'] = ((72 - X_test['tenure']) // 12) + 2014

# 月份衍生
X_train_seq['tenure_month'] = (72 - X_train['tenure']) % 12 + 1
X_test_seq['tenure_month'] = (72 - X_test['tenure']) % 12 + 1

# 季度衍生
X_train_seq['tenure_quarter'] = ((X_train_seq['tenure_month']-1) // 3) + 1
X_test_seq['tenure_quarter'] = ((X_test_seq['tenure_month']-1) // 3) + 1

# 独热编码
enc = preprocessing.OneHotEncoder()
enc.fit(X_train_seq)

seq_new = list(X_train_seq.columns)

# 创建带有列名称的独热编码之后的df
X_train_seq = pd.DataFrame(enc.transform(X_train_seq).toarray(), 
                           columns = cate_colName(enc, seq_new, drop=None))

X_test_seq = pd.DataFrame(enc.transform(X_test_seq).toarray(), 
                          columns = cate_colName(enc, seq_new, drop=None))

# 调整index
X_train_seq.index = X_train.index
X_test_seq.index = X_test.index

In [5]:
ord_enc = OrdinalEncoder()
ord_enc.fit(X_train[category_cols])

X_train_OE = pd.DataFrame(ord_enc.transform(X_train[category_cols]), columns=category_cols)
X_train_OE.index = X_train.index
X_train_OE = pd.concat([X_train_OE, X_train[numeric_cols]], axis=1)

X_test_OE = pd.DataFrame(ord_enc.transform(X_test[category_cols]), columns=category_cols)
X_test_OE.index = X_test.index
X_test_OE = pd.concat([X_test_OE, X_test[numeric_cols]], axis=1)

然后是模型融合部分所需的第三方库、准备的数据以及训练好的模型：

In [6]:
# 本节新增第三方库
from joblib import dump, load
from sklearn.ensemble import VotingClassifier
from hyperopt import hp, fmin, tpe
from numpy.random import RandomState
from sklearn.model_selection import cross_val_score

In [7]:
class VotingClassifier_threshold(BaseEstimator, ClassifierMixin, TransformerMixin):
    
    def __init__(self, estimators, voting='hard', weights=None, thr=0.5):
        self.estimators = estimators
        self.voting = voting
        self.weights = weights
        self.thr = thr
        
    def fit(self, X, y):
        VC = VotingClassifier(estimators = self.estimators, 
                              voting = self.voting, 
                              weights = self.weights)
        
        VC.fit(X, y)
        self.clf = VC
        
        return self
        
    def predict_proba(self, X):
        if self.voting == 'soft':
            res_proba = self.clf.predict_proba(X)
        else:
            res_proba = None
        return res_proba
    
    def predict(self, X):
        if self.voting == 'soft':
            res = (self.clf.predict_proba(X)[:, 1] >= self.thr) * 1
        else:
            res = self.clf.predict(X)
        return res
    
    def score(self, X, y):
        acc = accuracy_score(self.predict(X), y)
        return acc

In [8]:
# 实例化KFold评估器
kf = KFold(n_splits=5, random_state=12, shuffle=True)

# 重置训练集和测试集的index
X_train_OE = X_train_OE.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

train_part_index_l = []
eval_index_l = []

for train_part_index, eval_index in kf.split(X_train_OE, y_train):
    train_part_index_l.append(train_part_index)
    eval_index_l.append(eval_index)
    
# 训练集特征
X_train1 = X_train_OE.loc[train_part_index_l[0]]
X_train2 = X_train_OE.loc[train_part_index_l[1]]
X_train3 = X_train_OE.loc[train_part_index_l[2]]
X_train4 = X_train_OE.loc[train_part_index_l[3]]
X_train5 = X_train_OE.loc[train_part_index_l[4]]

# 验证集特征
X_eval1 = X_train_OE.loc[eval_index_l[0]]
X_eval2 = X_train_OE.loc[eval_index_l[1]]
X_eval3 = X_train_OE.loc[eval_index_l[2]]
X_eval4 = X_train_OE.loc[eval_index_l[3]]
X_eval5 = X_train_OE.loc[eval_index_l[4]]

# 训练集标签
y_train1 = y_train.loc[train_part_index_l[0]]
y_train2 = y_train.loc[train_part_index_l[1]]
y_train3 = y_train.loc[train_part_index_l[2]]
y_train4 = y_train.loc[train_part_index_l[3]]
y_train5 = y_train.loc[train_part_index_l[4]]

# 验证集标签
y_eval1 = y_train.loc[eval_index_l[0]]
y_eval2 = y_train.loc[eval_index_l[1]]
y_eval3 = y_train.loc[eval_index_l[2]]
y_eval4 = y_train.loc[eval_index_l[3]]
y_eval5 = y_train.loc[eval_index_l[4]]

train_set = [(X_train1, y_train1), 
             (X_train2, y_train2), 
             (X_train3, y_train3), 
             (X_train4, y_train4), 
             (X_train5, y_train5)]

eval_set = [(X_eval1, y_eval1), 
            (X_eval2, y_eval2), 
            (X_eval3, y_eval3), 
            (X_eval4, y_eval4), 
            (X_eval5, y_eval5)]

In [9]:
# 随机森林模型组
grid_RF_1 = load('./models/grid_RF_1.joblib') 
grid_RF_2 = load('./models/grid_RF_2.joblib') 
grid_RF_3 = load('./models/grid_RF_3.joblib') 
grid_RF_4 = load('./models/grid_RF_4.joblib') 
grid_RF_5 = load('./models/grid_RF_5.joblib') 

RF_1 = grid_RF_1.best_estimator_
RF_2 = grid_RF_2.best_estimator_
RF_3 = grid_RF_3.best_estimator_
RF_4 = grid_RF_4.best_estimator_
RF_5 = grid_RF_5.best_estimator_

RF_l = [RF_1, RF_2, RF_3, RF_4, RF_5]

# 决策树模型组
grid_tree_1 = load('./models/grid_tree_1.joblib')
grid_tree_2 = load('./models/grid_tree_2.joblib')
grid_tree_3 = load('./models/grid_tree_3.joblib')
grid_tree_4 = load('./models/grid_tree_4.joblib')
grid_tree_5 = load('./models/grid_tree_5.joblib')

tree_1 = grid_tree_1.best_estimator_
tree_2 = grid_tree_2.best_estimator_
tree_3 = grid_tree_3.best_estimator_
tree_4 = grid_tree_4.best_estimator_
tree_5 = grid_tree_5.best_estimator_

tree_l = [tree_1, tree_2, tree_3, tree_4, tree_5]

# 逻辑回归模型组
grid_lr_1 = load('./models/grid_lr_1.joblib')
grid_lr_2 = load('./models/grid_lr_2.joblib')
grid_lr_3 = load('./models/grid_lr_3.joblib')
grid_lr_4 = load('./models/grid_lr_4.joblib')
grid_lr_5 = load('./models/grid_lr_5.joblib')

lr_1 = grid_lr_1.best_estimator_
lr_2 = grid_lr_2.best_estimator_
lr_3 = grid_lr_3.best_estimator_
lr_4 = grid_lr_4.best_estimator_
lr_5 = grid_lr_5.best_estimator_

lr_l = [lr_1, lr_2, lr_3, lr_4, lr_5]

In [10]:
eval1_predict_proba_RF = pd.Series(RF_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index)
eval2_predict_proba_RF = pd.Series(RF_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index)
eval3_predict_proba_RF = pd.Series(RF_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index)
eval4_predict_proba_RF = pd.Series(RF_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index)
eval5_predict_proba_RF = pd.Series(RF_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index)

eval_predict_proba_RF = pd.concat([eval1_predict_proba_RF, 
                                   eval2_predict_proba_RF, 
                                   eval3_predict_proba_RF, 
                                   eval4_predict_proba_RF, 
                                   eval5_predict_proba_RF]).sort_index()

eval1_predict_proba_tree = pd.Series(tree_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index)
eval2_predict_proba_tree = pd.Series(tree_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index)
eval3_predict_proba_tree = pd.Series(tree_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index)
eval4_predict_proba_tree = pd.Series(tree_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index)
eval5_predict_proba_tree = pd.Series(tree_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index)

eval_predict_proba_tree = pd.concat([eval1_predict_proba_tree, 
                                     eval2_predict_proba_tree, 
                                     eval3_predict_proba_tree, 
                                     eval4_predict_proba_tree, 
                                     eval5_predict_proba_tree]).sort_index()

eval1_predict_proba_lr = pd.Series(lr_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index)
eval2_predict_proba_lr = pd.Series(lr_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index)
eval3_predict_proba_lr = pd.Series(lr_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index)
eval4_predict_proba_lr = pd.Series(lr_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index)
eval5_predict_proba_lr = pd.Series(lr_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index)

eval_predict_proba_lr = pd.concat([eval1_predict_proba_lr, 
                                   eval2_predict_proba_lr, 
                                   eval3_predict_proba_lr, 
                                   eval4_predict_proba_lr, 
                                   eval5_predict_proba_lr]).sort_index()

In [11]:
test_predict_proba_RF = []

for i in range(5):
    test_predict_proba_RF.append(RF_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_RF = np.array(test_predict_proba_RF)
test_predict_proba_RF = test_predict_proba_RF.mean(0)

test_predict_proba_tree = []

for i in range(5):
    test_predict_proba_tree.append(tree_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_tree = np.array(test_predict_proba_tree)
test_predict_proba_tree = test_predict_proba_tree.mean(0)

test_predict_proba_lr = []

for i in range(5):
    test_predict_proba_lr.append(lr_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_lr = np.array(test_predict_proba_lr)
test_predict_proba_lr = test_predict_proba_lr.mean(0)

## <center>Ch.3 模型融合基础方法

## 七、Stacking基本原理与实践

- Stacking方法模型融合方法快速尝试

&emsp;&emsp;在介绍完Voting和Averaging融合方法类之后，接下来还有一大类模型融合方法，也就是所谓的学习结合器类方法。

&emsp;&emsp;在Part 4.3.3的最后一部分，即围绕加权平均融合过程中，权重阈值的超参数搜索过程所遇到的过拟合问题，我们详细的讨论了相关解决方案，其中提出了一种用更加严谨的模型代替权重搜索这一解决问题的思路，其基本过程如下：

<center><img src="https://s2.loli.net/2022/05/22/eGR4Cl3dsFDpAES.png" alt="image-20220522174140320" style="zoom:33%;" />

实际上，在加权平均融合的过程中，我们是通过一个加权平均过程，配合阈值来完成融合，而该融合的本质，其实就是围绕单模输出结果（连续变量，每个样本预测为1的概率），来拟合预测标签（0-1二分类变量），因此其本质上就是一个分类问题，除了搜索权重和阈值外，我们也完全可以用机器学习的分类模型来完成这一预测过程。这里我们可以先迅速尝试使用逻辑回归模型来完成这一预测过程。

&emsp;&emsp;首先还是围绕我们训练好的三个模型，输出在训练集上的概率预测结果：

In [12]:
# 读取模型
logistic_search = load('./models/logistic_search.joblib') 
tree_model = load('./models/tree_model.joblib') 
RF_0 = load('./models/RF_0.joblib') 

# 训练集上的预测概率(预测为1的概率)
train_prediction1_proba = logistic_search.best_estimator_.predict_proba(X_train_OE)[:, 1]
train_prediction2_proba = tree_model.predict_proba(X_train_OE)[:, 1]
train_prediction3_proba = RF_0.predict_proba(X_train_OE)[:, 1]

# 测试集上的预测概率
test_prediction1_proba = logistic_search.best_estimator_.predict_proba(X_test_OE)[:, 1]
test_prediction2_proba = tree_model.predict_proba(X_test_OE)[:, 1]
test_prediction3_proba = RF_0.predict_proba(X_test_OE)[:, 1]

In [13]:
train_prediction1_proba

array([0.01069583, 0.56348576, 0.14572695, ..., 0.67546702, 0.05388848,
       0.00465571])

然后，以三个模型输出的概率预测结果作为训练数据，原数据集标签作为标签，进行逻辑回归模型训练。首先对原始的模型输出的概率结果进行拼接：

In [14]:
train_stack = np.vstack([[train_prediction1_proba], 
                         [train_prediction2_proba], 
                         [train_prediction3_proba]]).T

In [15]:
train_stack

array([[0.01069583, 0.02375297, 0.00672967],
       [0.56348576, 0.74912281, 0.51874357],
       [0.14572695, 0.22331155, 0.10493285],
       ...,
       [0.67546702, 0.45586298, 0.60729472],
       [0.05388848, 0.11384335, 0.0156605 ],
       [0.00465571, 0.02375297, 0.00535636]])

In [16]:
test_stack = np.vstack([[test_prediction1_proba], 
                        [test_prediction2_proba], 
                        [test_prediction3_proba]]).T

In [17]:
test_stack

array([[0.03485521, 0.02375297, 0.01044425],
       [0.2346808 , 0.11384335, 0.32168142],
       [0.00547175, 0.02375297, 0.00523215],
       ...,
       [0.13542354, 0.11384335, 0.15824046],
       [0.50080841, 0.45586298, 0.59381813],
       [0.06614044, 0.11384335, 0.10725971]])

In [18]:
y_train

0       0
1       0
2       0
3       0
4       0
       ..
5277    0
5278    0
5279    1
5280    0
5281    0
Name: Churn, Length: 5282, dtype: int64

然后带入模型进行训练，并输出准确率评分结果：

In [19]:
lr_final = LogisticRegression().fit(train_stack, y_train)
lr_final.score(train_stack, y_train), lr_final.score(test_stack, y_test)

(0.8782658084059068, 0.7898921067575241)

能够发现，逻辑回归模型能够顺利输出最终预测结果，模型融合能够顺利进行，而这种融合方法则被称为Stacking模型融合，该方法也是目前最为重要的一类模型融合方法，同时也是所谓学习结合器中效果最好的一类方法。当然，和其他模型融合方法类似，该方法在初次尝试的时候也遇到明显的过拟合现象，接下来，我们从更加严谨的角度介绍Stacking模型融合方法的基本原理，以及Stacking模型融合方法的优化策略。

### 1.Stacking模型融合基本原理与手动实现

&emsp;&emsp;和投票法&平局法类似，Stacking方法本质上也是一类结合方法，并与1992年由Wolpert等人提出，并在此后的数十年间不断完善与迭代，并且，该方法提出的核心目的同样也是为了有效结合多个弱学习器组成一个更强的学习器，但是，和投票法&平均法不同的是，Stacking是希望通过训练一个模型（而非找到某种加权平均的过程），来完成结合这一过程，在这个过程中，原始的个体学习器也被称为一级学习器，而结合过程用到的模型，则被称为二级学习器，或者元学习器。例如上述快速Stacking实践过程中，原始训练好的随机森林、逻辑回归和决策树三个模型就是一级学习器，而用于结合的逻辑回归模型则被称为元学习器（meta learner）。很明显，这是一种简单的双层级架构，而这种层级结构也就是Stacking（堆叠）名称的由来。Stacking中中元学习的训练和预测过程如下：

<center><img src="http://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20220919215031456.png" alt="image-20220522174140320" style="zoom:50%;" />

> 有时元学习器也被称为final model（最终学习器）。

&emsp;&emsp;当然，和其他结合方法一样，Stacking等学习结合器也有非常完整的基础理论，但同样，由于一级模型无法保证其独立性，因此大多数理论并不能很好的指导Stacking模型融合过程。此处我们先从中挑选部分有用的结论进行介绍，然后更多的从当前实践情况出发，介绍如何围绕Stacking融合过程进行优化。

- Stacking与Boosting

&emsp;&emsp;如果说投票法&均值法和Bagging的集成过程一致，那么Stacking则和Boosting算法有着千丝万缕的联系。从技术衍化发展角度来说，Stacking是Boosting的前身，Stacking为Boosting集成范式提供了非常多的基础理论支撑，可以说，正是由于Stacking方法的发明，才有了后续Boosting算法的突破；而从算法原理角度来说，Stacking和Boosting本质也是类似，Stacking也可以看成是是堆叠多层学习器，并通过不断拟合误差来提升效果。只不过相比之下Stacking作为结合方法，其过程会相对简单，适用的模型也更加广泛，而Boosting则是严谨的集成范式，只能借助某些某型来构建某种集成算法。

- 一级学习器的同质或异质性

&emsp;&emsp;根据Stacking的基本原理，一级学习器可以是相同的一组模型，此时便称一级学习器是同质的，反之，如果一级学习器是不同的模型，则称其为异质的。根据Stacking基本原理描述，一级学习器的同质或异质并不是影响Stacking结果的核心因素，但是，在一级学习器不独立的情况下，经过长期实践发现，异质的一级学习器要明显好于同质的一级学习器。因此，在实际建模过程中，更建议训练异质学习器。

- Stacking分类和回归

&emsp;&emsp;和其他模型融合方法一样，Stacking也同样可以处理回归问题或分类问题，只需要合理选择元学习器即可。例如，处理分类问题时，元学习器也对应选择分类问题模型，例如逻辑回归、分类树等；而处理回归问题时，元学习器则可以选择线性回归、贝叶斯回归、回归树等模型。当前案例是分类问题，我们将首先重点介绍分类问题的Stacking过程，后续还会有回归案例的讲解，届时将继续介绍Stacking解决回归问题的过程。

&emsp;&emsp;此外，更多的关于Stacking元学习器如何选择与优化，也将在后续Stacking优化部分内容一并讨论。

- Stacking过程中的“硬投票”或者“软投票”

&emsp;&emsp;其实，除了如上述代码展示的，可以带入一级学习器的概率预测结果到元学习器中参与训练（类似于软投票的过程），我们还可以以一级学习器输出的类别预测结果作为特征，带入到元学习器中进行训练（类似于硬投票的过程）。不过需要注意的是，无论是Stacking的相关理论（Ting & Witten[1999]），还是长期的实践征明，带入概率预测结果进行元学习器训练，不进行任何优化的情况下，和Stacking“软投票”效果不分伯仲，而如果带入某些优化方法（后续会介绍），则Stacking“软投票”效果会好很多。这里我们简单尝试Stacking“硬投票”：

In [20]:
# 训练集上的预测结果
train_prediction1 = logistic_search.best_estimator_.predict(X_train_OE)
train_prediction2 = tree_model.predict(X_train_OE)
train_prediction3 = RF_0.predict(X_train_OE)

# 测试集上的预测结果
test_prediction1 = logistic_search.best_estimator_.predict(X_test_OE)
test_prediction2 = tree_model.predict(X_test_OE)
test_prediction3 = RF_0.predict(X_test_OE)

In [21]:
train_prediction1

array([0, 1, 0, ..., 1, 0, 0])

In [22]:
train_stack_hard = np.vstack([[train_prediction1], 
                              [train_prediction2], 
                              [train_prediction3]]).T

In [23]:
train_stack_hard

array([[0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       ...,
       [1, 0, 1],
       [0, 0, 0],
       [0, 0, 0]], dtype=int64)

In [24]:
test_stack_hard = np.vstack([[test_prediction1], 
                             [test_prediction2], 
                             [test_prediction3]]).T

In [25]:
test_stack_hard

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [1, 0, 1],
       [0, 0, 0]], dtype=int64)

In [26]:
lr_final = LogisticRegression().fit(train_stack_hard, y_train)
lr_final.score(train_stack_hard, y_train), lr_final.score(test_stack_hard, y_test)

(0.8483528966300644, 0.7955706984667802)

能够发现，目前Stacking“硬投票”效果略好于“软投票”效果。但需要注意，当我们围绕Stacking过程进行优化时，限于“硬投票”的特征数值表现有限，Stacking“硬投票”效果将明显不如Stacking“软投票”过程，这点和加权平均融合过程类似。

> 当然，对于回归问题，并不会存在Stacking的“硬投票”过程，此时一级学习器输出的都是连续变量。

&emsp;&emsp;并且，我们不难发现，无论是带入类别预测结果还是概率结果，元学习器最终都表现出了一定程度的过拟合。这当然也是Stacking“理论缺陷”必然导致的结果，而如何优化，则是在执行整个Stacking融合过程中最核心的问题。在此前的模型融合的过程中，我们已经介绍了非常多优化策略，不过截至目前，能适用于Stacking融合过程的只有交叉训练这一种方法。

- Stacking一级学习器的交叉训练过程

&emsp;&emsp;在介绍更多Stacking优化方法之前，我们先探讨如何借助交叉训练来抑制Stacking过拟合。

&emsp;&emsp;实际上，早在1998年，Smyth & Wolpert等人就从理论角度提出，交叉训练能够有效提升元学习器的泛化能力，其实相比投票法&均值法，交叉训练更像是为Stacking融合过程量身定制的方法，时至今日，交叉训练更是已经成为Stacking过程的“标配”，包括sklearn在内的各机器学习算法库，也是将Stacking模型融合与交叉训练捆绑在一起来进行实现，以提升Stacking效果。

&emsp;&emsp;在Part 4.3.4中，我们已经详细介绍了交叉训练过程，而在Stacking中调用交叉训练过程只会影响元学习器特征（train_stack & test_stack）创建过程：即不再是简单输出训练集上概率预测结果作为训练集和测试集特征，而是用验证集拼接后的数据集作为train_stack，而用测试集的平均概率预测结果作为test_stack。以五折随机森林模型交叉训练为例，该过程创建的某条特征（注意是一个模型创建一个特征）过程如下：

<center><img src="http://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20220920160449348.png" alt="image-20220920160449348" style="zoom:50%;" />

其中具体的代码实现过程在Part 4.3.4中有详细介绍，此处不做赘述，目前我们创建的eval_predict_proba_RF对象就是上图中的train_stack_RF，而test_predict_proba_RF则是上图中的test_stack_RF。而三个模型分别交叉训练得到的特征，拼接而成的数据集，就将是最终用于训练元学习器的数据集train_stack:

In [27]:
eval_predict_proba_RF

0       0.044787
1       0.572187
2       0.161815
3       0.250871
4       0.122533
          ...   
5277    0.082653
5278    0.346562
5279    0.551481
5280    0.049011
5281    0.002783
Length: 5282, dtype: float64

In [28]:
# 元学习器训练集
train_stack = pd.DataFrame({'train_stack_RF': eval_predict_proba_RF, 
                            'train_stack_lr': eval_predict_proba_lr, 
                            'train_stack_tree': eval_predict_proba_tree})

train_stack

,train_stack_RF,train_stack_lr,train_stack_tree
0,0.044787,0.011289,0.037669
1,0.572187,0.542331,0.787986
2,0.161815,0.154121,0.222819
3,0.250871,0.273393,0.259434
4,0.122533,0.158399,0.107345
...,...,...,...
5277,0.082653,0.083575,0.062959
5278,0.346562,0.365228,0.222819
5279,0.551481,0.674365,0.438538
5280,0.049011,0.050536,0.066419


In [29]:
# 元学习器测试集
test_stack = pd.DataFrame({'test_stack_RF': test_predict_proba_RF, 
                           'test_stack_lr': test_predict_proba_lr, 
                           'test_stack_tree': test_predict_proba_tree})

test_stack

,test_stack_RF,test_stack_lr,test_stack_tree
0,0.029220,0.045946,0.046473
1,0.311980,0.233711,0.158900
2,0.016244,0.005192,0.046473
3,0.025769,0.035878,0.046473
4,0.035476,0.056142,0.051573
...,...,...,...
1756,0.193367,0.169545,0.212513
1757,0.048821,0.040112,0.046473
1758,0.145346,0.125325,0.158900
1759,0.530686,0.501991,0.437401


接下来尝试带入训练元学习器：

In [30]:
lr_final = LogisticRegression().fit(train_stack, y_train)
lr_final.score(train_stack, y_train), lr_final.score(test_stack, y_test)

(0.8178720181749337, 0.7955706984667802)

能够发现，模型过拟合现象得到了有效抑制。不难理解，对于Stacking来说，同样是因为交叉训练能够非常好的做到信息隔离，从而能够一定程度提高元学习的泛化能力。在没借助其他优化方法的情况下，Stacking能够有如此效果，也足见Stacking过程本身的威力。

> 注，不同模型验证集拼接成的训练集很多时候也被标注为oof，意为out-of-fold predictions，验证集预测结果。

至此，我们就完整实现的手动Stacking的全过程。接下来我们进一步介绍如何借助sklearn来进行Stacking模型融合。

### 2.Stacking模型融合的sklearn实现过程

#### 2.1 评估器参数介绍与调用过程

&emsp;&emsp;自0.22版本开始，sklearn也集成了Stacking模型融合相关功能，并支持自动交叉训练和“软硬投票”等各种功能。这里首先导入分类问题的Stacking融合评估器：

In [31]:
from sklearn.ensemble import StackingClassifier

In [32]:
StackingClassifier?

Init signature:
StackingClassifier(
    estimators,
    final_estimator=None,
    *,
    cv=None,
    stack_method='auto',
    n_jobs=None,
    passthrough=False,
    verbose=0,
)
Docstring:     
Stack of estimators with a final classifier.

Stacked generalization consists in stacking the output of individual
estimator and use a classifier to compute the final prediction. Stacking
allows to use the strength of each individual estimator by using their
output as input of a final estimator.

Note that `estimators_` are fitted on the full `X` while `final_estimator_`
is trained using cross-validated predictions of the base estimators using
`cross_val_predict`.

Read more in the :ref:`User Guide <stacking>`.

.. versionadded:: 0.22

Parameters
----------
estimators : list of (str, estimator)
    Base estimators which will be stacked together. Each element of the
    list is defined as a tuple of string (i.e. name) and an estimator
    instance. An estimator can be set to 'drop' using `set_p

该评估器的核心参数解释如下：

|参数|解释|
|:--:|:--:|
|estimators|一级评估器|
|final_estimator|二级评估器，默认是逻辑回归|
|cv|一级评估器基交叉训练折数|
|stack_method|选择概率结果还是类别结果进行元学习器的训练|
|passthrough|是否额外带入原始数据特征进行元学习器的训练|

其中estimators参数结构和投票法评估器结构一致，都是需要创建一个由（模型名称、模型）所组成的一个列表。例如，当我们采用逻辑回归、决策树和随机森林三个模型进行Stacking模型融合时，estimators应该按照如下格式进行创建：

In [33]:
estimators = [('lr', logistic_search.best_estimator_), ('tree', tree_model), ('rf', RF_0)]

而final_estimator就是元学习器，只需要实例化一个sklearn中的评估器即可；CV就是一级评估器交叉训练的折数，默认是五折。        
&emsp;&emsp;stack_method则可以选择元学习器的训练数据类型，可选'auto'、'predict_proba'、'decision_function'、'predict'四个不同取值。当输入'predict_proba'时，即带入样本类别概率进行训练，而'decision_function'则是SVM特殊的一种模型输出结果，代表样本到分割超平面的（置信）距离，同样也可以充当类似概率的作用，距离越短，则模型判断越不肯定（相当于概率越趋近于0.5），而'predict'则是样本类别结果，相当于是Stacking“硬投票”，在默认情况下，参数选择为'aotu'，即根据不同模型，按照'predict_proba'>'decision_function'>'predict'的优先级进行参数选择。当然，对于逻辑回归、决策树和随机森林来说，参数输入aotu时就是根据预测概率训练元学习器。

> 同时，根据sklearn的相关说明，当stack_method选择'predict_proba'时，元学习器会带入预测为1的概率进行计算，这一点和手动实现过程完全一致。

&emsp;&emsp;最后一个参数passthrough，也是最为特殊的一个参数，那就是是否额外带入原始数据集特征进行元学习器训练，默认参数是False：只带入一级学习器的预测结果训练元学习器。但当我们选择参数为True时，会拼凑一个由一级学习器输出结果和原始特征共同拼接而成的数据集，用于元学习器的训练。该操作本质上其实是一种特征增强方法，常常用于层级堆叠结构的模型训练过程，包括某些Boosting、深度森林的级联训练等，都有可能用到特征增强技术。

&emsp;&emsp;但是，特征增强本身是一项较为复杂的技术，其实现过程非常灵活，可以在一个堆叠的模型结构的任意层增加任意任意特征，甚至是衍生特征，但目前来看，除了深度森林和Blending（一种模型融合方法）给出了明确的能提升效果的特征增强方法外，其他场景的特征增强方法效果都不确定，也就是说需要反复多次尝试，来找到可能能提升效果的特征增强方法。基于此，sklearn的Stacking评估器中passthrough参数取值，也是需要多加尝试的，并非一定带入或者不带入原始数据集特征就能获得更好的效果。

&emsp;&emsp;更多关于特征增强的相关方法介绍，我们将在模型融合部分内容结束后详细讨论。

- Stacking评估器调用

&emsp;&emsp;接下来，我们尝试调用sklearn中Stacking评估器来执行模型融合：

In [34]:
# 实例化Stacking评估器
# 元学习器选择逻辑回归模型，这也是默认参数
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

In [35]:
clf.fit(X_train_OE, y_train)

StackingClassifier(estimators=[('lr',
                                Pipeline(steps=[('columntransformer',
                                                 ColumnTransformer(transformers=[('cat',
                                                                                  OneHotEncoder(drop='if_binary'),
                                                                                  ['gender',
                                                                                   'SeniorCitizen',
                                                                                   'Partner',
                                                                                   'Dependents',
                                                                                   'PhoneService',
                                                                                   'MultipleLines',
                                                                                   'InternetService',
   

对于Stacking评估器，我们也可以非常方便的查看一级、二级学习器的基本情况：

In [36]:
# 一级学习器
clf.estimators

[('lr',
  Pipeline(steps=[('columntransformer',
                   ColumnTransformer(transformers=[('cat',
                                                    OneHotEncoder(drop='if_binary'),
                                                    ['gender', 'SeniorCitizen',
                                                     'Partner', 'Dependents',
                                                     'PhoneService',
                                                     'MultipleLines',
                                                     'InternetService',
                                                     'OnlineSecurity',
                                                     'OnlineBackup',
                                                     'DeviceProtection',
                                                     'TechSupport', 'StreamingTV',
                                                     'StreamingMovies',
                                                     'Contract',
      

In [37]:
clf.estimators[1]

('tree',
 DecisionTreeClassifier(ccp_alpha=0, max_depth=5, max_leaf_nodes=8,
                        random_state=12))

In [38]:
# 二级学习器调用
clf.final_estimator

LogisticRegression()

In [39]:
# 查看训练完成后二级学习器的参数
clf.final_estimator.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

以及其他的一些关键参数：

In [40]:
clf.stack_method

'auto'

In [41]:
clf.passthrough

False

然后查看模型融合性能：

In [42]:
clf.score(X_train_OE, y_train), clf.score(X_test_OE, y_test)

(0.8273381294964028, 0.787052810902896)

能够发现，尽管都是5折交叉训练，尽管都是默认参数的逻辑回归作为元学习器，尽管都是相同的一级学习器，Stacking评估器的性能要弱于手动实现的Stacking过程。这是为什么呢？究其原因还是因为Stacking评估器中训练的多组模型，每一组模型的超参数是固定的，并不能像手动实现过程那样非常精细的去给每一组的每一个模型进行超参数优化。很明显，每一组模型共用一组超参数，还是会一定程度影响融合性能。

&emsp;&emsp;当然，我们还可以尝试进行Stacking“硬投票”，即选择stack_method为predict：

In [43]:
clf = StackingClassifier(estimators=estimators, stack_method='predict').fit(X_train_OE, y_train)

In [44]:
clf.stack_method

'predict'

In [45]:
clf.score(X_train_OE, y_train), clf.score(X_test_OE, y_test)

(0.8290420295342673, 0.7859170925610448)

能够发现，在基于交叉训练的Stacking融合过程中，“软投票”过程要好于“硬投票”过程。

#### 2.2 Stacking评估器优化

&emsp;&emsp;尽管sklearn中Stacking评估器的适用不如手动实现Stacking过程那么灵活，但作为sklearn内部的评估器，是可以非常便捷的调用sklearn中其他功能，以此来快速实现诸如级联优化、多层Stacking等更复杂的过程。而这些其实也都是后续我们将要详细介绍的Stacking优化的核心方法。

&emsp;&emsp;因此，我们先借助sklearn的Stacking评估器来进行一些优化实验和尝试，在得到一些基本结论与经验后，我们再通过手动实现的方式，进行更加精准深度的优化。

- Stacking评估器内部的超参数优化

&emsp;&emsp;这里首先我们尝试着对Stacking评估器内部的一些超参数进行优化。主要是三个核心参数，分别是cv、stack_method和passthrough。这三个参数取值的组合有限，我们可以借助网格搜索对其完成最佳超参数搜索：

In [46]:
start = time.time()

# 设置超参数空间
parameter_space = {"cv": range(1, 11), 
                   "stack_method": ['predict_proba', 'decision_function', 'predict'],
                   "passthrough": [True, False]}

# 实例化Stacking评估器
clf = StackingClassifier(estimators=estimators)
stack_grid = GridSearchCV(clf, parameter_space, n_jobs=15)

# 模型训练
stack_grid.fit(X_train_OE, y_train)

print(time.time()-start)

34.35744905471802


In [47]:
stack_grid.best_params_

{'cv': 2, 'passthrough': False, 'stack_method': 'predict_proba'}

In [48]:
stack_grid.best_score_

0.8116245233794903

In [49]:
stack_grid.score(X_train_OE, y_train), stack_grid.score(X_test_OE, y_test)

(0.8256342294585385, 0.7876206700738216)

能够发现，经过网格搜索后，确实得到了一组测试集上表现更好的一组超参数。passthrough和stack_method参数的最佳取值和默认参数一致（其中stack_method为auto时也就是predict_proba），而CV的最佳取值是2，可能和“交叉验证折数越高、泛化能力越强”这一判断有悖。但实际上，在样本量较少的时候折数设置更小往往可能会有更好的效果，这就类似软投票过程中进行阈值移动时采用的三折验证而不是五折验证。

&emsp;&emsp;当然，具体几折验证，其实很大程度也会参与Stacking的模型相关，模型不同、训练方式不同，最佳折数也会随之发生变化。这里我们重点需要强调的是passthrough和stack_method两个参数的取值选取，一般来说，尤其是在调用sklearn进行Stacking的时候，往往不不带入原始数据、只带入一级模型输出的概率预测结果，进行元学习器的训练，能得到一个更好的结果。

- 元学习器模型选择

&emsp;&emsp;对于一个层级结构融合过程来说，每个阶段的模型肯定是对最终结果有着至关重要的影响。接下来我们进一步在网格搜索中加入不同元学习器进行筛选，看是否会获得一个更好的Stacking效果：

In [76]:
start = time.time()

# 设置超参数空间
clf1 = DecisionTreeClassifier()
clf2 = LogisticRegression()
clf3 = RandomForestClassifier()

parameter_space = {"cv": range(1, 11), 
                   "stack_method": ['predict_proba', 'decision_function', 'predict'],
                   "final_estimator": [clf1, clf2, clf3],
                   "passthrough": [True, False]}

# 实例化Stacking评估器
clf = StackingClassifier(estimators=estimators)
stack_grid = GridSearchCV(clf, parameter_space, n_jobs=15)

# 模型训练
stack_grid.fit(X_train_OE, y_train)

print(time.time()-start)

96.94286322593689


In [77]:
stack_grid.best_params_

{'cv': 2,
 'final_estimator': LogisticRegression(),
 'passthrough': False,
 'stack_method': 'predict_proba'}

In [78]:
stack_grid.best_score_

0.8116245233794903

In [79]:
stack_grid.score(X_train_OE, y_train), stack_grid.score(X_test_OE, y_test)

(0.8256342294585385, 0.7876206700738216)

能够发现，相比决策树和随机森林，逻辑回归表现更好。其实在Stacking的元学习器的选择中，并不是越复杂的模型效果越好，而是视具体情况而定，一般来说，如果一级学习器比此独立性越强，则元学习器学习能力越强越好，但如果一级学习器彼此独立型较弱，则元学习器则需要具备一定的天然抗过拟合特性，才能够获得一个更好的输出结果。当前的一级学习器尽管是经过了交叉训练，但sklearn自带交叉训练过程并不能在超参数层面进行信息隔离，因此实际效力有限，外加一级学习器都是在相同的数据集上进行模型训练，因此独立性是有限的，此时就需要一个抗过拟合较好的元学习器。而对于逻辑回归模型来说，首先其学习能力有限，其次sklearn的逻辑回归模型自带正则化项，因此在默认参数设置情况下，就能一定程度规避过拟合问题，因此该模型也是当前情况下拟合效果最好的元学习器。

> 类似的，如果是回归类问题，在一级学习器独立性较弱的情况下，元学习器选择贝叶斯回归、Lasso、岭回归等模型效果会比较好。

> 尽管逻辑回归并不是预测能力最强的模型，但逻辑回归其实用途很广，除了作为为数不多的机器学习领域的可解释型模型，同时还在模型融合、推荐系统中数据编码等领域发挥着重要作用，在Kaggle举办的2021年最受欢迎模型评选中，逻辑回归更是名列榜首。这也是为何课程中会花费大量篇幅来介绍逻辑回归模型的原因。

- 元学习器手动超参数优化

&emsp;&emsp;而既然更好的抗过拟合特性能够帮助元学习提高融合效果，那么对于逻辑回归模型来说，我们将惩罚项手动调整为l1正则化，或许能够提升Stacking融合效果：

In [109]:
final_lr = LogisticRegression(penalty='l1', solver='saga')

In [110]:
clf = StackingClassifier(estimators=estimators, final_estimator=final_lr).fit(X_train_OE, y_train)

In [111]:
clf.score(X_train_OE, y_train), clf.score(X_test_OE, y_test)

(0.8271488072699735, 0.7893242475865985)

> 这里需要注意，惩罚项调整为l1正则化后，优化器也需要调整为saga，否则损失函数无法求解。关于逻辑回归的参数讲解，可以回顾Lesson 6的相关内容。

不出所料，最终融合效果有所提升。当然，对于其他模型来说，通过超参数的设计调整，也是能提升模型抗过拟合特性的。例如在决策树模型设置最大树深度为2，通过降低模型复杂度来提升模型抗过拟合特性：

In [125]:
final_tree = DecisionTreeClassifier(max_depth=2)

In [126]:
clf = StackingClassifier(estimators=estimators, final_estimator=final_tree).fit(X_train_OE, y_train)

In [127]:
clf.score(X_train_OE, y_train), clf.score(X_test_OE, y_test)

(0.8131389625141991, 0.7904599659284497)

能够发现，融合效果确实有所提升，该思路确实有效。

- 元学习器自动超参数搜索

&emsp;&emsp;不过，尽管我们发现降低元学习器的复杂度、提升元学习器的抗过拟合特性，能够提升Stacking融合效果，但手动的超参数调整终究效果有限，能否借助优化器来进行元学习器的超参数优化呢？

&emsp;&emsp;方法是有的，不过首先我们需要明确的是，我们无法在Stacking评估器层面对元学习器的超参数进行搜索，Stacking评估器层面只能优化Stacking评估器自己的超参数。要解决这个问题，我们必须把元学习器和Stacking评估器串联起来，然后联动调参，即主要调整元学习器的超参数，然后根据Stacking输出作为反馈，来不断修正元学习器的最佳超参数取值。很明显，这个过程需要修改Stacking评估器。

&emsp;&emsp;此处选取决策树模型作为元学习器，来尝试进行Stacking的元学习器超参数优化：

In [50]:
class Stacking_tree_Cascade(BaseEstimator, ClassifierMixin, TransformerMixin):
    
    def __init__(self, estimators, cv=None, passthrough='auto', max_depth=None, min_samples_split=2, min_samples_leaf=1, max_leaf_nodes=None):
        self.estimators = estimators
        self.cv = cv
        self.passthrough = passthrough
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_leaf_nodes = max_leaf_nodes
        self.final_estimator = DecisionTreeClassifier(max_depth=self.max_depth, 
                                                      min_samples_split=self.min_samples_split, 
                                                      min_samples_leaf=self.min_samples_leaf, 
                                                      max_leaf_nodes=self.max_leaf_nodes)
        
    def fit(self, X, y):
        SC = StackingClassifier(estimators = self.estimators, 
                                final_estimator = self.final_estimator,
                                cv = self.cv, 
                                passthrough = self.passthrough)

        SC.fit(X, y)
        self.clf = SC
        self.classes_ = pd.Series(y).unique()
        return self

    def predict_proba(self, X):
        res_proba = self.clf.predict_proba(X)
        return res_proba

    def predict(self, X):
        res = self.clf.predict(X)
        return res

    def score(self, X, y):
        acc = accuracy_score(self.predict(X), y)
        return acc

然后测试支持元学习器超参数优化的Stacking评估器能否正常运行：

In [51]:
STC = Stacking_tree_Cascade(estimators).fit(X_train_OE, y_train)

In [52]:
STC.score(X_train_OE, y_train), STC.score(X_test_OE, y_test)

(0.8112457402499054, 0.7387847813742192)

此时元学习器的超参数就也可以作为外层评估器的超参数进行调整：

In [53]:
STC = Stacking_tree_Cascade(estimators, max_depth=2).fit(X_train_OE, y_train)

In [54]:
STC.score(X_train_OE, y_train), STC.score(X_test_OE, y_test)

(0.8131389625141991, 0.7904599659284497)

In [75]:
start = time.time()

parameter_space =  {'max_depth': np.arange(2, 7, 1).tolist(), 
                    'min_samples_split': np.arange(2, 7, 1).tolist(), 
                    'min_samples_leaf': np.arange(2, 7, 1).tolist(), 
                    'max_leaf_nodes':np.arange(4, 10, 1).tolist(), 
                    'cv':np.arange(2, 6, 1).tolist()}

# 实例化Stacking评估器
STC = Stacking_tree_Cascade(estimators)
STC_grid = GridSearchCV(STC, parameter_space, n_jobs=15)

# 模型训练
STC_grid.fit(X_train_OE, y_train)

print(time.time()-start)

1507.0625259876251


In [76]:
STC_grid.best_params_

{'cv': 4,
 'max_depth': 3,
 'max_leaf_nodes': 8,
 'min_samples_leaf': 6,
 'min_samples_split': 3}

In [77]:
STC_grid.score(X_train_OE, y_train), STC_grid.score(X_test_OE, y_test)

(0.8118137069291935, 0.7887563884156729)

&emsp;&emsp;至此，我们就完成了一次简单的元学习器超参数搜索优化实践。当然，元学习器的优化对于Stacking最终建模效果影响非常巨大，更多的元学习器的选择与优化策略将在下一小节进行详细讨论。

- 多层Stacking

&emsp;&emsp;当然，除了元学习器优化外，Stacking的另一个重要的优化方向就是多层Stacking。

&emsp;&emsp;其实从此前的介绍不难看出，Stacking的本质就是围绕上一层模型输出结果进行学习，借此提升最终预测效果。而在这个过程中，元学习器本身也是可以输出概率预测结果的，也就是说，某个元学习器之后还可以再堆叠一层元学习器。而如果两层的堆叠能够提升单模效果，那么双层的堆叠则能够进一步提升学习能力，从而进一步提升模型效果。当然，伴随着Stacking结构更加复杂，融合的过拟合风险也会更高。本节我们先讨论如何实现多层的Stacking，下一小节开始将重点介绍如何优化。

&emsp;&emsp;例如，我们可以借助逻辑回归、决策树和随机森林三个模型，构建如下三层Stacking堆叠融合：

<center><img src="http://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20220920225821282.png" alt="image-20220920225821282" style="zoom:50%;" />

在上述三层评估器堆叠的情况下，两个一级元学习器接受的数据是相同的，而各自的预测结果则会为二级元学习器提供一个特征。该过程的实现代码如下：

In [79]:
# 先实例化一级元学习器
tree_final = DecisionTreeClassifier()
RF_final = RandomForestClassifier()

# 然后构建两层元学习器之间的Stacking评估器
final_layer = StackingClassifier(estimators=[('tree_final', tree_final), ('RF_final', RF_final)], 
                                 final_estimator=LogisticRegression(penalty='l1', solver='saga'))

# 然后构建一级学习器
multi_layer = StackingClassifier(estimators=estimators, final_estimator=final_layer)

然后进行训练：

In [80]:
multi_layer.fit(X_train_OE, y_train)

StackingClassifier(estimators=[('lr',
                                Pipeline(steps=[('columntransformer',
                                                 ColumnTransformer(transformers=[('cat',
                                                                                  OneHotEncoder(drop='if_binary'),
                                                                                  ['gender',
                                                                                   'SeniorCitizen',
                                                                                   'Partner',
                                                                                   'Dependents',
                                                                                   'PhoneService',
                                                                                   'MultipleLines',
                                                                                   'InternetService',
   

查看最终模型训练结果：

In [81]:
multi_layer.score(X_train_OE, y_train), multi_layer.score(X_test_OE, y_test)

(0.8275274517228323, 0.7893242475865985)

至此，我们在借助sklearn的情况下，快速构建了一个简易的多层Stacking融合器，不过在大多数情况下，多层Stacking、或者是不同种类融合混合堆叠，其实都会有较为严重的过拟合问题。除非是非常特殊的数据情况，否则一般来说多层融合只有一个应用场景：那就是Blending+均值法两层融合。这方面内容我们将在Part 4.3.10中进行详细的探讨。

&emsp;&emsp;至此，我们就完整介绍了Stacking的原理和基本实践过程，并简单尝试了部分优化策略。其中有较为进阶的优化方法如特征增强、多层模型融合等，此外也有较为基础但往往行之有效的优化方法，即一级学习器的训练流程优化与元学习器优化。那么从下一小节开始，我们将由浅入深，从Stacking基础优化方法开始，逐步介绍学习器结合器的实战优化策略。